# NMSU CSCI-5435 Assignment 4 Task 1 - LSTM

## Relevent Information

In [1]:
#Name:               Tianjie Chen
#Email:              tvc5586@nmsu.edu
#File Creation Date: Mar/17/2025
#Purpose of File:    NMSU CSCI-5435 Assignment 4 Task 1
#Last Edit Date:     Mar/17/2025
#Last Edit Note:     File creation
#GenAI used:         False

## Load Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import gensim.downloader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.autograd import Variable

## Setup

In [3]:
# USING GPU
print(torch.cuda.device_count())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1


In [4]:
batch_size = 128  # BATCH SIZE FOR THIS MODEL
epochs     = 5   # Number of training epochs

In [5]:
DATA_PATH = "News_Category_Dataset_v2.json"

train_data = pd.read_json(DATA_PATH, lines = True)['short_description']

## Preprocessing

In [6]:
train_data = train_data.str.lower()
train_data = train_data.str.replace("-", " ", regex=True)
train_data = train_data.str.replace(r"[^'\&\w\s]", "", regex=True)
train_data = train_data.str.strip()
train_data = [" ".join(["<start>", x, "<end>"]) for x in train_data]

In [7]:
print(train_data[0])

<start> she left her husband he killed their children just another day in america <end>


## LSTM

In [8]:
from transformers import BertTokenizerFast

###
# define Vocab
###
class Vocab:
    def __init__(self, list_of_sentence, tokenization, special_token, max_tokens=None):
        # count vocab frequency
        vocab_freq = {}
        tokens = tokenization(list_of_sentence)
        for t in tokens:
            for vocab in t:
                if vocab not in vocab_freq:
                    vocab_freq[vocab] = 0
                vocab_freq[vocab] += 1
        # sort by frequency
        vocab_freq = {k: v for k, v in sorted(vocab_freq.items(), key=lambda i: i[1], reverse=True)}
        # create vocab list
        self.vocabs = special_token + list(vocab_freq.keys())
        if max_tokens:
            self.vocabs = self.vocabs[:max_tokens]
        self.stoi = {v: i for i, v in enumerate(self.vocabs)}

    def _get_tokens(self, list_of_sentence):
        for sentence in list_of_sentence:
            tokens = tokenizer.tokenize(sentence)
            yield tokens

    def get_itos(self):
        return self.vocabs

    def get_stoi(self):
        return self.stoi

    def append_token(self, token):
        self.vocabs.append(token)
        self.stoi = {v: i for i, v in enumerate(self.vocabs)}

    def __call__(self, list_of_tokens):
        def get_token_index(token):
            if token in self.stoi:
                return self.stoi[token]
            else:
                return 0
        return [get_token_index(t) for t in list_of_tokens]

    def __len__(self):
        return len(self.vocabs)

###
# generate Vocab
###
max_word = 50000

# create tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Must manually add the start and end tokens, otherwise
# the tokenizer will separate them into three tokens
tokenizer.add_tokens(["<start>", "<end>"])

# define tokenization function
def yield_tokens(data):
    for text in data:
        tokens = tokenizer.tokenize(text)
        yield tokens

# build vocabulary list
vocab = Vocab(
    train_data,
    tokenization=yield_tokens,
    special_token=["<unk>"],
    max_tokens=max_word,
)

# get list for index-to-word, and word-to-index.
itos = vocab.get_itos()
stoi = vocab.get_stoi()

# Add <pad> token
vocab.append_token("<pad>")

#### Configure Model

In [9]:
class LSTM_LM(nn.Module):
    def __init__(self, vocab_size, seq_len, embedding_dim, rnn_units, padding_idx):
        super().__init__()

        self.seq_len = seq_len
        self.padding_idx = padding_idx

        self.embedding = nn.Embedding(
            vocab_size,
            embedding_dim,
            padding_idx=padding_idx,
        )
        self.LSTM = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=rnn_units,
            num_layers=1,
            batch_first=True,
        )
        self.classify = nn.Linear(rnn_units, vocab_size)

    def forward(self, inputs, states=None, return_final_state=False):
        # embedding
        #   --> (batch_size, seq_len, embedding_dim)
        outs = self.embedding(inputs)
        # build "lengths" property to pack inputs (see above)
        lengths = (inputs != self.padding_idx).int().sum(dim=1, keepdim=False)
        # pack inputs for RNN
        packed_inputs = torch.nn.utils.rnn.pack_padded_sequence(
            outs,
            lengths.cpu(),
            batch_first=True,
            enforce_sorted=False,
        )
        # apply RNN
        if states is None:
            packed_outs, final_state = self.LSTM(packed_inputs)
        else:
            packed_outs, final_state = self.LSTM(packed_inputs, states)
        # unpack results
        #   --> (batch_size, seq_len, rnn_units)
        outs, _ = torch.nn.utils.rnn.pad_packed_sequence(
            packed_outs,
            batch_first=True,
            padding_value=0.0,
            total_length=self.seq_len,
        )
        # apply feed-forward to classify
        #   --> (batch_size, seq_len, vocab_size)
        logits = self.classify(outs)
        # return results
        if return_final_state:
            return logits, final_state  # This is used in prediction
        else:
            return logits               # This is used in training

#### Train Model

In [10]:
embedding_dim = 64
rnn_units = 512
max_seq_len = 256
pad_index = vocab.__len__() - 1

In [11]:
def collate_batch(batch):
    label_list, feature_list = [], []
    for text in batch:
        # tokenize to a list of word's indices
        tokens = vocab(tokenizer.tokenize(text))
        # separate into features and labels
        y = tokens[1:]
        y.append(-100)
        x = tokens
        # limit length to max_seq_len
        y = y[:max_seq_len]
        x = x[:max_seq_len]
        # pad features and labels
        y += [-100] * (max_seq_len - len(y))
        x += [pad_index] * (max_seq_len - len(x))
        # add to list
        label_list.append(y)
        feature_list.append(x)
    # convert to tensor
    label_list = torch.tensor(label_list, dtype=torch.int64).to(device)
    feature_list = torch.tensor(feature_list, dtype=torch.int64).to(device)
    return label_list, feature_list

dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_batch
)

In [12]:
model = LSTM_LM(
    vocab_size=vocab.__len__(),
    seq_len=max_seq_len,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    padding_idx=pad_index).to(device)

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

for epoch in range(epochs):
    for labels, seqs in dataloader:
        # optimize
        optimizer.zero_grad()
        logits = model(seqs.to(device))
        loss = F.cross_entropy(logits.transpose(1,2), labels.to(device))
        loss.backward()
        optimizer.step()
        # calculate accuracy
        pred_labels = logits.argmax(dim=2)
        num_correct = (pred_labels == labels).float().sum()
        num_total = (labels != -100).float().sum()
        accuracy = num_correct / num_total
        print("Epoch {} - loss: {:2.4f} - accuracy: {:2.4f}".format(epoch+1, loss.item(), accuracy), end="\r")
    print("")

Epoch 1 - loss: 5.6789 - accuracy: 0.1353
Epoch 2 - loss: 5.2175 - accuracy: 0.2144
Epoch 3 - loss: 4.7239 - accuracy: 0.2272
Epoch 4 - loss: 4.9198 - accuracy: 0.1968
Epoch 5 - loss: 4.8169 - accuracy: 0.2237


#### Text Generation

In [14]:
end_index = stoi["<end>"]
max_output = 128

def pred_output(text):
    generated_text = "<start> " + text
    _, inputs = collate_batch([generated_text])
    mask = (inputs != pad_index).int()
    last_idx = mask[0].sum() - 1
    final_states = None
    outputs, final_states = model(inputs, final_states, return_final_state=True)
    pred_index = outputs[0][last_idx].argmax()
    for loop in range(max_output):
        generated_text += " "
        next_word = itos[pred_index]
        generated_text += next_word
        if pred_index.item() == end_index:
            break
        _, inputs = collate_batch([next_word])
        outputs, final_states = model(inputs, final_states, return_final_state=True)
        pred_index = outputs[0][0].argmax()
    return generated_text

In [15]:
print(pred_output("in the united states president"))
print(pred_output("the man has accused by"))
print(pred_output("now he was expected to"))

<start> in the united states president o ##ba ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es ##s ##es
<start> the man has accused by the us president elect <end>
<start> now he was expected to be a woman who has been a long history of the <end>
